<a href="https://colab.research.google.com/github/BlackJack2021/Zenn_abe_2020_description/blob/main/Abe(2020).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データの取得・整形

データは CDNOW データセットを用いる。データは [こちらからダウンロード](https://www.brucehardie.com/datasets/) できるものを利用する。


In [2]:
!wget https://www.brucehardie.com/datasets/CDNOW_master.zip
!unzip CDNOW_master.zip

--2024-02-03 05:35:47--  https://www.brucehardie.com/datasets/CDNOW_master.zip
Resolving www.brucehardie.com (www.brucehardie.com)... 66.96.146.129
Connecting to www.brucehardie.com (www.brucehardie.com)|66.96.146.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 433231 (423K) [application/zip]
Saving to: ‘CDNOW_master.zip’

CDNOW_master.zip    100%[===================>] 423.08K  --.-KB/s    in 0.1s    

2024-02-03 05:35:47 (3.67 MB/s) - ‘CDNOW_master.zip’ saved [433231/433231]

Archive:  CDNOW_master.zip
  inflating: CDNOW_master.txt        
  inflating: read_me_CDNOW_master.txt  


In [3]:
import pandas as pd
import warnings

warnings.simplefilter('ignore')

# テキストファイルを読み込む
# 区切り文字をスペースとして指定し、列名を明示的に設定
raw_df = pd.read_csv('CDNOW_master.txt', sep='\s+', names=['customer_id', 'date', 'quantity', 'price'])
df = raw_df.copy()

# 日付データを datetime 型に変更
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# 売上 (total_amount) 変数を作成
df['total_amount'] = df['quantity'] * df['price']

print('各データの型')
print(df.dtypes)
print('\nサンプル数')
print(len(df))
print('\n例')
print(df.head())

各データの型
customer_id              int64
date            datetime64[ns]
quantity                 int64
price                  float64
total_amount           float64
dtype: object

サンプル数
69659

例
   customer_id       date  quantity  price  total_amount
0            1 1997-01-01         1  11.77         11.77
1            2 1997-01-12         1  12.00         12.00
2            2 1997-01-12         5  77.00        385.00
3            3 1997-01-02         2  20.76         41.52
4            3 1997-03-30         2  20.76         41.52


同一の日付に複数購入がされているデータも見られるので、同一の日付における複数の購入は1レコードにまとめてしまう（おそらく、同一の日付に違う商品が購入された場合に複数レコード登場してしまっているのだと推測される）。

我々の分析でほしいのは、各日付の総支払額なので、それを求めた `agg_df` を定義する。


In [4]:
agg_df = df.groupby(['customer_id', 'date'])['total_amount'].sum().reset_index()

print('各データの型')
print(agg_df.dtypes)
print('\nサンプル数')
print(len(agg_df))
print('\n例')
print(agg_df.head())

各データの型
customer_id              int64
date            datetime64[ns]
total_amount           float64
dtype: object

サンプル数
67591

例
   customer_id       date  total_amount
0            1 1997-01-01         11.77
1            2 1997-01-12        397.00
2            3 1997-01-02         41.52
3            3 1997-03-30         41.52
4            3 1997-04-02         39.08


# 顧客データの選別

ここでは Abe(2020) の選択基準と同じように、5回以上の購買があった顧客にデータを絞り込んで分析を行うことにする。


In [5]:
customer_count_df = agg_df.groupby('customer_id').size().reset_index(name='count')
candidate_customers = customer_count_df[customer_count_df['count'] >= 5]['customer_id'].tolist()
print('元々のユーザー数', len(customer_count_df))
print('5回以上の購買ユーザー数', len(candidate_customers))

filtered_df = agg_df[agg_df['customer_id'].isin(candidate_customers)]

元々のユーザー数 23570
5回以上の購買ユーザー数 3835


したがって、今回の分析に利用する顧客の数は3835人である。

# 尤度関数の記述に向けて変数抽出

同梱されている説明文には以下のような記述がある。

> 1997年第1四半期にCDNOWで初めて購入した23,570人のコホート全員の、**1998年6月末までの全購入履歴**が含まれています。このCDNOWデータセットは、FaderとHardie（2001）によって初めて使用されました。このファイルの各レコードには、69,659件のレコードがあり、顧客のID、取引の日付、購入されたCDの数、取引のドル価値の4つのフィールドが含まれています。

つまり、このデータにおける観測期間の打ち切りは `1998-06-30` である。あるユーザーの購入時点が $(y_1, y_2,...,y_n)$ と表現されると仮定する。この時、尤度関数の記述のために必要な変数は以下である。

1. 購買間隔日数 $(t_1, t_2,...,t_{n-1})$
    - ただし、$t_1 \equiv y_2 - y_1$, $t_2 \equiv y_3 - y_2$,... である。
2. 最終購買時点から観測打ち切り時点までの経過日数($r$): いわゆるRecency
    - $1998年6月30日 - y_n$ で計算できる
3. 各購買における支払い金額 $(s_1, s_2,..., s_n)$
4. 観測期間開始から打ち切りまでの経過時間: $T$

まず、特定の顧客のデータ: `pd.DataFrame` が与えられたときに、そのデータからこれらの情報を抽出する関数を用意する。


In [23]:
from typing import List, TypedDict

class Features(TypedDict):
    '''尤度関数の記述に必要な変数を格納する辞書の型を定義

    Args:
        intervals(List[float]): 購買間隔日数のリスト
        recency(float): 最終購買時点から観測打ち切り時点までの経過日数
        expenditures(List[float]) 各購買における支払い金額
        T(float): 観測期間開始から打ち切りまでの経過時間
    '''
    intervals: List[int]
    recency: float
    expenditures: List[float]
    T: float

class CustomerFeatures(TypedDict):
    customer_id: int
    features: Features

def extract_features(customer_id: int, customer_df: pd.DataFrame) -> CustomerFeatures:
    '''分析対象のユーザーのデータから、尤度の記述に必要な変数を取得します

    Args:
        df(pd.DataFrame): 以下の列が含まれるものです。
            customer_id: int64
            date: datetime64[ns]
            total_amount: float64
    '''

    def _extract_intervals(customer_df: pd.DataFrame) -> List[int]:
        '''各購買の間隔を計算'''
        intervals = customer_df['date'].diff().dt.days.tolist()
        # 先頭が nan になるので取り除く
        return intervals[1:]

    def _extract_recency(customer_df: pd.DataFrame) -> int:
        '''最終購買時点から観測打ち切り時点までの経過日数を計算'''
        observation_end = pd.to_datetime('1998-06-30')
        last_purchase = customer_df['date'].max()
        return (observation_end - last_purchase).days

    def _extract_expenditures(customer_df: pd.DataFrame) -> List[float]:
        '''各時点の購買金額を取得'''
        return customer_df['total_amount'].tolist()

    def _extract_T(customer_df: pd.DataFrame) -> int:
        '''最初の購買が発生してから打ち切りまでの経過時間'''
        observation_end = pd.to_datetime('1998-06-30')
        observation_start = customer_df['date'].min()
        return (observation_end - observation_start).days

    intervals = _extract_intervals(customer_df)
    recency = _extract_recency(customer_df)
    expenditures = _extract_expenditures(customer_df)
    T = _extract_T(customer_df)

    result: CustomerFeatures = {
        'customer_id': customer_id,
        'features': {
            'intervals': intervals ,
            'recency': recency,
            'expenditures': expenditures,
            'T': T
        }
    }

    return result

上記のコードを利用して、各顧客ごとの特徴量を抽出し、List 形式で全て格納

In [24]:
from tqdm import tqdm
features_of_customers: List[CustomerFeatures] = []
for customer_id, customer_df in tqdm(filtered_df.groupby('customer_id')):
    features = extract_features(customer_id, customer_df)
    features_of_customers.append(features)

100%|██████████| 3835/3835 [00:07<00:00, 511.85it/s]


In [25]:
from pprint import pprint
pprint(features_of_customers[:3])

[{'customer_id': 3,
  'features': {'T': 544,
               'expenditures': [41.52, 41.52, 39.08, 287.25, 83.84, 16.99],
               'intervals': [87.0, 3.0, 227.0, 10.0, 184.0],
               'recency': 33}},
 {'customer_id': 5,
  'features': {'T': 545,
               'expenditures': [58.66,
                                13.97,
                                116.69999999999999,
                                136.64999999999998,
                                116.13,
                                52.28,
                                56.28,
                                121.41,
                                185.84,
                                121.41,
                                112.41],
               'intervals': [13.0,
                             21.0,
                             66.0,
                             50.0,
                             16.0,
                             36.0,
                             55.0,
                             84.0,


購買金額に関する推定には以下を用いる。

$$
\begin{align}
\hat{log(m)} = \frac{1}{n}\sum_{i=1}^{n}\log(s_n) \\
\omega^2 = \frac{1}{n-1}\sum_{i=1}^{n}\biggl[ \log(s_i) - \hat{log(m)} \biggr]^2
\end{align}
$$

次に残された $a, b, \mu$ については以下で示される対数尤度を最大化するように推定する。

$$
\begin{align}
\log(L(a, b, \mu)) = \sum_{i=1}^{n-1}\log(g(t_i)) - \mu T + \log\bigg\{ [1-G(r)] + e^{\mu r} \int_{0}^{r} \frac{\mu e^{-\mu x}}{1+e^{-a+bx}} dx  \biggr\}
\end{align}
$$

しかしこの対数尤度関数は複雑な形状であり、最尤推定量を解析的に得ることが困難であるので、数値解析により尤度を最大化する推定量を算出する。

# 実装

以下で阿部モデルの推定を行うクラス `AbeModel` を実装する。($a, b, \mu$) についての推定には `scypy.optimize` が提供する `minimize` 関数を利用して、対数尤度の最大化問題を数値解析で解く。利用している手法は `L-BFGS-B` である。



In [27]:
import numpy as np
from scipy.integrate import quad
from typing import Union
from scipy.optimize import minimize, Bounds

class InitialGuess(TypedDict):
    a: float
    b: float
    mu: float

class Estimation(TypedDict):
    customer_id: int
    log_m: float
    omega_sq: float
    a: Optional[float]
    b: Optional[float]
    mu: Optional[float]
    success: bool
    message: Optional[str]

class AbeModel:
    def __init__(self):
        '''各パラメータを保存するためのプロパティを作成'''
        self.a: float = 1.0
        self.b: float = 1.0
        self.mu: float = 0.1
        self.log_m: float = 1.0
        self.omega_sq: float = 1.0


    def _fit_log_m(self, expenditures: np.ndarray) -> float:
        '''log_m の最尤推定を実施'''
        return np.mean(np.log(expenditures))

    def _fit_omega_sq(self, expenditures: np.ndarray, log_m: float) -> float:
        '''omega^2 の不偏推定量を計算'''
        degree_of_freedom = len(expenditures) - 1
        return np.sum(np.square(np.log(expenditures) - log_m)) / degree_of_freedom

    def _calc_G(self, intervals: Union[np.ndarray, float]) -> Union[np.ndarray, float]:
        '''Gを計算'''
        return np.exp(-self.a + self.b * intervals) / (1 + np.exp(-self.a + self.b * intervals))

    def _calc_g(self, intervals: Union[np.ndarray, float]) -> Union[np.ndarray, float]:
        '''gを計算'''
        G = self._calc_G(intervals)
        # gの計算で0になるのを防ぐためにepsilonを追加。ここが0になると log(g) の計算が不適切になる。
        epsilon = 1e-10
        g = self.b * (1 - G) * G
        # gが非常に小さい値（epsilon以下）の場合は、epsilonを返す
        g_with_min_value = np.maximum(g, epsilon)
        return g_with_min_value

    def _llh_first_term(self, intervals: np.ndarray) -> float:
        '''$\sum_{i=1}^{n-1}\log{(g(t_i))}$ を計算'''
        log_gs = np.log(self._calc_g(intervals))
        return np.sum(log_gs)

    def _llh_second_term(self, T: float) -> float:
        return - self.mu * T

    def _integral_term(self, r: float) -> float:
        '''$\int_{0}^{r}\frac{\mu e^{-\mu x}}{1 + e^{-a+bx}}dx$ を近似計算'''
        def integrand(x: float):
            return (self.mu * np.exp(-self.mu * x)) / (1 + np.exp(-self.a + self.b * x))
        approx_value, _ = quad(integrand, 0, r)
        return approx_value

    def _llh_third_term(self, r: float) -> float:
        '''対数尤度の第三項を計算'''
        value = 1 - self._calc_G(r) + np.exp(self.mu * r) * self._integral_term(r)
        return np.log(value)

    def _calc_llh(self, intervals: np.ndarray, T: float, r: float) -> float:
        '''(a, b, mu) に関する対数尤度を計算'''
        log_likelihood = (
            self._llh_first_term(intervals) + self._llh_second_term(T) + self._llh_third_term(r)
        )
        return log_likelihood

    def _neg_llh(self, params: np.ndarray, intervals: np.ndarray, T: float, r: float) -> float:
        '''負の対数尤度関数を計算する'''
        self.a, self.b, self.mu = params  # パラメータを更新
        return -self._calc_llh(intervals, T, r)  # 対数尤度の負の値

    def fit(self, intervals: np.ndarray, expenditures: np.ndarray, T: float, r: float, initial_guess: InitialGuess) -> Estimation:
        '''最尤推定によるパラメータの推定'''
        # まずは解析解が得られるものを計算
        self.log_m = self._fit_log_m(expenditures)
        self.omega_sq = self._fit_omega_sq(expenditures, self.log_m)

        # 続いて数値解析でパラメータを推定
        initial_guess_list = [initial_guess['a'], initial_guess['b'], initial_guess['mu']]  # 初期値
        bounds = Bounds([-np.inf, 1e-10, 0], [np.inf, np.inf, np.inf])  # パラメータの最小値と最大値を指定

        result = minimize(self._neg_llh, initial_guess_list, args=(intervals, T, r), method='L-BFGS-B', bounds=bounds)
        self.a, self.b, self.mu = result.x
        return {
            'log_m': self.log_m,
            'omega_sq': self.omega_sq,
            'a': self.a,
            'b': self.b,
            'mu': self.mu,
            'success': result.success,
            'message': result.message
        }

In [26]:
from typing import Optional

estimations: List[Estimation] = []
for i in tqdm(range(len(features_of_customers))):
    sample = features_of_customers[i]
    customer_id = sample['customer_id']
    intervals = np.array(sample['features']['intervals'])
    expenditures = np.array(sample['features']['expenditures'])
    T = sample['features']['T']
    r = sample['features']['recency']
    initial_guess = {'a': 1, 'b': 1, 'mu': 0.1}
    model = AbeModel()
    estimation = model.fit(intervals, expenditures, T, r, initial_guess)
    estimation['customer_id'] = customer_id
    estimations.append(estimation)

estimation_df = pd.DataFrame(estimations)

100%|██████████| 3835/3835 [01:17<00:00, 49.62it/s]


In [31]:
estimation_df.head()

,log_m,omega_sq,a,b,mu,success,message,customer_id
0,4.006642,0.913002,1.933006,0.018158,0.000000,True,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,3
1,4.424795,0.516846,4.249037,0.061470,0.006147,True,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,5
2,3.789849,1.801814,2.842791,0.033968,0.000000,True,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,8
3,3.094266,0.643223,1.605469,0.023422,0.000000,True,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,25
4,4.321208,0.957014,2.071638,0.048349,0.000000,True,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,29
